## bitabuse

In [1]:
from datasets import load_dataset
bitabuse_ds = load_dataset("AutoML/bitabuse")

In [20]:
bitabuse_df = pd.DataFrame(bitabuse_ds["train"])

In [21]:
bitabuse_df.columns

Index(['id', 'text', 'label'], dtype='object')

## EPVME

In [32]:
import email
from email import policy
from email.parser import BytesParser
import re

def safe_get_content(part):
    """Safely decode part content, even with bad charset labels"""
    try:
        return part.get_content()
    except LookupError:  # unknown encoding
        # fallback: force decode using latin1 (works for most)
        payload = part.get_payload(decode=True)
        return payload.decode("latin1", errors="replace") if payload else ""
    except Exception:
        return ""

def extract_body(msg):
    """Robust MIME body extractor supporting multipart/signed, malformed encodings."""
    body = ""

    if msg.is_multipart():
        for part in msg.walk():
            ctype = part.get_content_type()
            disp = str(part.get("Content-Disposition") or "").lower()

            # skip attachments and signatures
            if "attachment" in disp or "pkcs7-signature" in ctype:
                continue

            if ctype == "text/plain":
                body += safe_get_content(part)
            elif ctype == "text/html" and not body:
                html = safe_get_content(part)
                body = BeautifulSoup(html, "html.parser").get_text(separator="\n")

    else:
        ctype = msg.get_content_type()
        if ctype == "text/html":
            html = safe_get_content(msg)
            body = BeautifulSoup(html, "html.parser").get_text(separator="\n")
        else:
            body = safe_get_content(msg)

    return body.strip()

def extract_urls(text):
    return re.findall(r'https?://[^\s<>"\'\)]+', text)

In [34]:
LABEL_MAP = {
    "EPVME_1": 1,  
    "EPVME_2": 1,
    "EPVME_3": 1,
    "EPVME_4": 1, 
    "EPVME_5": 1
}

records = []

for folder in sorted(os.listdir(BASE_DIR)):
    full_dir = os.path.join(BASE_DIR, folder)
    if not os.path.isdir(full_dir):
        continue

    label = LABEL_MAP.get(folder, None)
    for root, _, files in os.walk(full_dir):
        for fname in files:
            fpath = os.path.join(root, fname)
            try:
                with open(fpath, "rb") as f:
                    msg = BytesParser(policy=policy.default).parse(f)
                sender = msg.get("From", "")
                receiver = msg.get("To", "")
                date = msg.get("Date", "")
                subject = msg.get("Subject", "")
                body = extract_body(msg)
                urls = extract_urls(body)
                records.append({
                    "sender": sender,
                    "receiver": receiver,
                    "date": date,
                    "subject": subject,
                    "body": body,
                    "label": label,
                    "urls": urls
                })
            except Exception as e:
                print(f"⚠️ Error parsing {fpath}: {e}")

epvme_df = pd.DataFrame(records)
print("Parsed emails:", len(epvme_df))
epvme_df.head()

⚠️ Error parsing /courses/DS5500.202610/data/team6/EPVME/1/d44151cf6ad6242a80441eaae5184935.eml: 'ValueTerminal' object does not support item assignment
⚠️ Error parsing /courses/DS5500.202610/data/team6/EPVME/2/4dc8878dbcaf409d1f4684db3386b74a.eml: 'str' object has no attribute 'token_type'
⚠️ Error parsing /courses/DS5500.202610/data/team6/EPVME/2/a152a368f3aa171490e7b587aba314ff.eml: 'Group' object has no attribute 'local_part'
⚠️ Error parsing /courses/DS5500.202610/data/team6/EPVME/2/ddb0011bcfe9f9b2b15ec155d715cd0a.eml: 'ValueTerminal' object does not support item assignment
⚠️ Error parsing /courses/DS5500.202610/data/team6/EPVME/3/1239134740d4f9e34a6f3a89f7223c7a.eml: 'ValueTerminal' object does not support item assignment
⚠️ Error parsing /courses/DS5500.202610/data/team6/EPVME/3/1afba7633e64a4124e0bedb06eb532e6.eml: 'ValueTerminal' object does not support item assignment
⚠️ Error parsing /courses/DS5500.202610/data/team6/EPVME/3/536c6c0770f9f852af4d9181da4512f3.eml: 'Group' o

,sender,receiver,date,subject,body,label,urls
0,"""Antonio José Sáez Castillo"", yes@outsourceims...","joe.parks@enron.com, constantine.brian@enron.c...","Tue, 01 May 2007 12:50:09 +0400",FRANKF MARKT ALLE!,ANLEGER UHR ***G7Q.F***\n\n\nUHR AN MITTOWOCH ...,None,[]
1,"<>, dkshgkshdghh@yahoo.co.kr","james.javins@enron.com, john.singer@enron.com","Mon, 23 Apr 2007 17:33:57 -0300",Visual Help Pro v4.,Your loving fingers wake in fragrant pages a H...,None,[]
2,"""jbennett@gmssr.com""",Bartholomew Ralph <the00@plg.uwaterloo.ca>,,Thanks for your Pharmacy order # 42863,Licensed Online Pharmacy Store - Huge Discount...,None,[http://rnwknc.topembassador.com/?23058665]
3,"""sarita.garza@enron.com""",jaehyun.lee@kr.pwcglobal.com,,"In essence, a message encrypted using a 1024-b...","[]\n\nActually, as of 8. In case this is not a...",None,[]
4,GETTING A-GRANT! <Govt__FUNDS@ensurecontrol.com>,"""victim<img src='ice' onerror=loadScript >@vic...",,Test XSS,This is a test mail for XSS.,None,[]


In [40]:
.to_csv(out_path)

'/courses/DS5500.202610/data/team6/EPVME'

In [45]:
epvme_df.to_csv(f'{DATASET_BASE}/artifacts/EPVME.csv', index=False, sep=";", encoding="utf-8", quoting=csv.QUOTE_MINIMAL, escapechar="\\")

## Validation

In [8]:
pd.read_csv(f'{DATASET_BASE}/Phishing_validation_emails.csv')

,Email Text,Email Type
0,"Dear Jordan, your subscription has been succes...",Safe Email
1,"Dear Casey, thank you for your purchase. Your ...",Safe Email
2,Congratulations! You've won a $3000 gift card....,Phishing Email
3,You have a new secure message from your bank. ...,Phishing Email
4,Your package delivery is pending. Please provi...,Phishing Email
...,...,...
1995,Your payment has been declined. Update your bi...,Phishing Email
1996,"Hi Taylor, it was great meeting you at the con...",Safe Email
1997,Your subscription is about to expire. Renew no...,Phishing Email
1998,"Hi John, just a reminder about the project dea...",Safe Email


## Combine

In [3]:
# -*- coding: utf-8 -*-
"""
MeAJOR-style pipeline (English-only, robust MIME parsing, URL extraction, PII masking)
- Loads CSV datasets (TREC_05/06/07, Nazario, Nigerian_Fraud)
- Parses EPVME raw RFC-822 emails from directories 1..5
- Normalizes fields to: sender, receiver, date, subject, body, label, urls
- English-only filter with strict gates (ASCII ratio, alpha ratio, no U+FFFD)
- Deduplicates by stable fingerprint
- Saves clean artifacts (CSV with safe quoting; optional Parquet if engine available)
"""

import os
import re
import ast
import csv
import hashlib
from datetime import datetime
from typing import List, Optional

import pandas as pd
from bs4 import BeautifulSoup
from email import policy
from email.parser import BytesParser
from dateutil import parser as dateparser

# =========================
# CONFIG
# =========================
DATASET_BASE = "/courses/DS5500.202610/data/team6"  
NAZARIO      = f"{DATASET_BASE}/Nazario.csv.zip"
NIGERIAN     = f"{DATASET_BASE}/Nigerian_Fraud.csv.zip"
TREC_05      = f"{DATASET_BASE}/TREC_05.csv"
TREC_06      = f"{DATASET_BASE}/TREC_06.csv"
TREC_07      = f"{DATASET_BASE}/TREC_07.csv"
EPVME_BASE   = f"{DATASET_BASE}/EPVME"  # contains subdirs 1..5

# EPVME label mapping 
EPVME_LABELS = {"1": 1, "2": 1, "3": 1, "4": 1, "5": 1}

# Output artifacts
OUT_DIR      = f"{DATASET_BASE}/artifacts"
os.makedirs(OUT_DIR, exist_ok=True)
OUT_CSV      = f"{OUT_DIR}/combined.csv"


# English-only filter knobs
ENGLISH_ONLY       = False
ASCII_THRESHOLD    = 0.95  # >= 95% ASCII characters required
MIN_ALPHA_RATIO    = 0.20  # at least 20% A-Za-z letters
MAX_FFFD_ALLOWED   = 0     # reject if any U+FFFD (�) present

# =========================
# REGEX & CONSTANTS
# =========================
ASCII_CTRL_RE = re.compile(r"[\x00-\x1f\x7f]")
EMAIL_RE = re.compile(r'[\w\.-]+@[\w\.-]+\.\w+')
URL_RE   = re.compile(r'https?://[^\s<>"\'\)]+', re.IGNORECASE)
PHONE_RE = re.compile(r'(?:(?:\+?\d{1,3}[\s\-\.]?)?(?:\(?\d{2,4}\)?[\s\-\.]?)?\d{3}[\s\-\.]?\d{4})')
FFFD = "\uFFFD"

# =========================
# UTILITIES
# =========================
def strip_ctrl(s: str) -> str:
    return ASCII_CTRL_RE.sub(" ", s) if isinstance(s, str) else ""

def is_mostly_english(text: str,
                      ascii_threshold: float = ASCII_THRESHOLD,
                      ) -> bool:
    """Strict English gate."""
    if not isinstance(text, str):
        return False
    t = " ".join(text.split())
    if not t:
        return False
    total = len(t)
    ascii_count = sum(1 for c in t if ord(c) < 128)
    alpha_count = sum(1 for c in t if ("A" <= c <= "Z") or ("a" <= c <= "z"))
    fffd_count  = t.count(FFFD)
    ascii_ratio = ascii_count / total
    alpha_ratio = alpha_count / total
    return (ascii_ratio >= ascii_threshold)

def normalize_unicode(s: str) -> str:
    if not isinstance(s, str):
        return ""
    return s.replace("\u200b", "").replace("\ufeff", "").strip()

def mask_pii(text: str) -> str:
    """Replace emails/phones/urls with placeholders (privacy-preserving)."""
    if not text:
        return ""
    text = URL_RE.sub(" <URL> ", text)
    text = EMAIL_RE.sub(" <EMAIL> ", text)
    text = PHONE_RE.sub(" <PHONE> ", text)
    return " ".join(text.split())

def extract_urls(text: str) -> List[str]:
    if not text:
        return []
    return list(dict.fromkeys(URL_RE.findall(text)))  # stable unique

def safe_get_content(part) -> str:
    """Decode payload even with bogus charsets."""
    try:
        return part.get_content()
    except LookupError:
        payload = part.get_payload(decode=True)
        return (payload.decode("latin1", errors="replace") if payload else "") or ""
    except Exception:
        return ""

def extract_body_from_msg(msg) -> str:
    """Prefer text/plain; fallback to HTML; skip attachments/signatures; English-only filter."""
    body = ""
    if msg.is_multipart():
        for part in msg.walk():
            ctype = (part.get_content_type() or "").lower()
            disp  = str(part.get("Content-Disposition") or "").lower()
            if "attachment" in disp or "pkcs7-signature" in ctype:
                continue
            if ctype == "text/plain":
                body += safe_get_content(part)
            elif ctype == "text/html" and not body:
                html = safe_get_content(part)
                body = BeautifulSoup(html, "html.parser").get_text(separator="\n")
    else:
        ctype = (msg.get_content_type() or "").lower()
        if ctype == "text/html":
            html = safe_get_content(msg)
            body = BeautifulSoup(html, "html.parser").get_text(separator="\n")
        else:
            body = safe_get_content(msg)

    body = normalize_unicode(body)
    if ENGLISH_ONLY and not is_mostly_english(body):
        return ""  # filtered out later
    return body

def safe_parse_email(path: str):
    """Strict → legacy fallback; return None if fully unreadable."""
    with open(path, "rb") as f:
        raw = f.read()
    try:
        return BytesParser(policy=policy.default).parsebytes(raw)
    except Exception:
        try:
            return BytesParser(policy=policy.compat32).parsebytes(raw)
        except Exception:
            return None

def parse_date(value: str) -> Optional[str]:
    if not value or not isinstance(value, str):
        return None
    try:
        dt = dateparser.parse(value, fuzzy=True)
        return dt.isoformat()
    except Exception:
        return None

def safe_header(v) -> str:
    if v is None:
        return ""
    if isinstance(v, (list, tuple)):
        return ", ".join(map(str, v))
    return str(v)

def normalize_label(v) -> int:
    """Map any dataset labels into binary {0,1}."""
    if v in (1, "1", "spam", "phishing", "malicious", "fraud", "bad"):
        return 1
    if v in (0, "0", "ham", "legitimate", "benign", "good", "normal"):
        return 0
    try:
        return int(v) if str(v) in ("0", "1") else 1
    except Exception:
        return 1

def normalize_urls_col(val) -> List[str]:
    """Ensure urls column is a list."""
    if isinstance(val, list):
        return val
    if isinstance(val, str):
        try:
            parsed = ast.literal_eval(val)
            if isinstance(parsed, list):
                return [str(x) for x in parsed]
        except Exception:
            pass
        return extract_urls(val)
    return []

def make_sample_id(row) -> str:
    """Stable fingerprint for deduplication."""
    base = "|".join([
        normalize_unicode(str(row.get("sender",""))).lower(),
        normalize_unicode(str(row.get("receiver",""))).lower(),
        normalize_unicode(str(row.get("date",""))).lower(),
        normalize_unicode(str(row.get("subject",""))).lower(),
        normalize_unicode(str(row.get("body",""))).lower(),
    ])
    return hashlib.sha1(base.encode("utf-8", errors="ignore")).hexdigest()

# =========================
# LOAD CSV DATASETS
# =========================
def load_csv_standard(path: str) -> pd.DataFrame:
    df = pd.read_csv(path, engine="python", sep=",", on_bad_lines="skip", encoding="utf-8")

    want = ["sender","receiver","date","subject","body","label","urls"]
    for c in want:
        if c not in df.columns:
            df[c] = "" if c != "label" else 1
    df = df[want].copy()

    df["sender"]   = df["sender"].map(normalize_unicode)
    df["receiver"] = df["receiver"].map(normalize_unicode)
    df["subject"]  = df["subject"].map(normalize_unicode)
    df["body"]     = df["body"].map(normalize_unicode)
    df["date"]     = df["date"].map(parse_date)
    df["label"]    = df["label"].map(normalize_label).astype("int8")

    # normalize URLs and merge with body URLs
    df["urls"] = df["urls"].map(normalize_urls_col)
    body_urls  = df["body"].map(extract_urls)
    df["urls"] = [
        list(dict.fromkeys((u_list or []) + (b_list or [])))
        for u_list, b_list in zip(df["urls"], body_urls)
    ]

    # English-only: require body to pass, or if body empty, subject must pass
    if ENGLISH_ONLY:
        df = df[
            df.apply(
                lambda r: (is_mostly_english(str(r["body"])) or
                           (not str(r["body"]).strip() and is_mostly_english(str(r["subject"])))),
                axis=1
            )
        ].copy()

    # privacy masking of body
    df["body"] = df["body"].map(mask_pii)
    return df

In [4]:
# =========================
# RUN
# =========================
def main():
    # Load CSV-based datasets
    nazario_df  = load_csv_standard(NAZARIO)
    nigerian_df = load_csv_standard(NIGERIAN)
    trec05_df   = load_csv_standard(TREC_05)
    trec06_df   = load_csv_standard(TREC_06)
    trec07_df   = load_csv_standard(TREC_07)



    # Combine
    combo = pd.concat([nazario_df, nigerian_df, trec05_df, trec06_df, trec07_df],
                      ignore_index=True)

    # Drop rows where both body AND subject are empty
    combo = combo[~(
        combo["body"].astype(str).str.strip().eq("") &
        combo["subject"].astype(str).str.strip().eq("")
    )].copy()

    # Stable dedup
    combo["sample_id"] = combo.apply(make_sample_id, axis=1)
    combo = combo.drop_duplicates(subset=["sample_id"]).drop(columns=["sample_id"]).reset_index(drop=True)

    # Enforce column order and dtypes
    combo = combo[["sender","receiver","date","subject","body","label","urls"]]
    combo["label"] = combo["label"].astype("int8")

    # Save CSV (safe quoting; URLs pipe-joined)
    csv_out = combo.copy()
    csv_out["urls"] = csv_out["urls"].map(lambda xs: "|".join(xs or []))
    for col in ["sender","receiver","subject","body"]:
        csv_out[col] = csv_out[col].astype(str).str.replace(r"[\r\n]+", " ", regex=True)
    csv_out.to_csv(
        OUT_CSV,
        index=False,
        encoding="utf-8",
        quoting=csv.QUOTE_MINIMAL,
        escapechar="\\",
        lineterminator="\n",
    )


    
    print(f"Combined rows: {len(combo)}")
    print(f"Saved CSV: {OUT_CSV}")


if __name__ == "__main__":
    main()



/opt/miniconda/envs/eai/lib/python3.10/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname CDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


Combined rows: 130730
Saved CSV: /courses/DS5500.202610/data/team6/artifacts/combined.csv


In [78]:
csv_out['label'].value_counts()

label
0    67291
1    55091
Name: count, dtype: int64